In [1]:
import numpy as np
from astropy.io import fits

import subprocess

In [2]:
#First, separate the images. 

reg = (slice(40,900), slice(300,1700))

#Image with the better seeing. 
h = fits.open("science_reduced_img.W0116-0505.60148.I_BESS.chip1.9.bkg.fits")
fits.writeto("work/t.fits"   , h[0].data[reg], overwrite=True)
fits.writeto("work/terr.fits", h[1].data[reg], overwrite=True)
h.close()

#Now the image with the worse seeing. 
h = fits.open("science_reduced_img.W0116-0505.60201.I_BESS.chip1.2.bkg.fits")
fits.writeto("work/im.fits"   , h[0].data[reg], overwrite=True)
fits.writeto("work/imerr.fits", h[1].data[reg], overwrite=True)
h.close()

We need to run Imagematch in the forwards mode, not reverse. The reason is that if ran into reverse mode (-R option), the registered convolved image is not saved, only the convolved image before registration is. To create the 2D maps, we need the images to be properly registered. 

Because of this, our "science" image is the one with the worse seeing (im.fits, the one we are trying to match to), and our "reference" image is the one with the better seeing (t.fits, the one we are trying to blur and register to match im.fits). 

In [3]:
command = "ImageMatch -v -preserve -cfg sample.cfg -m work/t.fits -sig err work/im.fits"
print(command)
subprocess.call(command, shell=True, executable='/bin/zsh', stdout=subprocess.DEVNULL)

ImageMatch -v -preserve -cfg sample.cfg -m work/t.fits -sig err work/im.fits


/Users/rjassef/miniconda3/envs/stenv/lib/python3.11/site-packages/imagematch/ImageMatching_scalerot.py:1471: RuntimeWarning: invalid value encountered in sqrt
  self.csigma = np.sqrt(self.csigma)


0

Now, we need to recover the noise image. We have that imdiff_sigma.fits is the error in the difference image, and is its the combination in quadrature of the error in imerr.fits (the "science" image) and the error in the convolved and registered "reference" image, which is what we are trying to recover.

In [4]:
imerr = fits.getdata("work/imerr.fits")
imdifferr = fits.getdata("work/imdiff_sigma.fits")

terr_blurred = (imdifferr**2 - imerr**2)**0.5
fits.writeto("work/terr_blurred.fits", terr_blurred, overwrite=True)